In [1]:
# para no usar GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf

2025-05-16 11:21:28.017198: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-16 11:21:28.017297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-16 11:21:28.018413: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-16 11:21:28.023609: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-16 11:21:28.698177: W tensorflow/compiler/tf2

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# lista de acciones y precio de petroleo y oro
tickers = ['GOOG', 'AMZN', 'META', 'TSLA', 'AAPL', 'MSFT', 'NVDA', 'CL=F', 'GC=F']
data = yf.download(tickers, start='2010-01-01', end='2024-12-31', auto_adjust=False)['Adj Close']
data = data.ffill().bfill()

[*********************100%***********************]  9 of 9 completed


In [3]:
data

Ticker,AAPL,AMZN,CL=F,GC=F,GOOG,META,MSFT,NVDA,TSLA
Date,,,,,,,,,
2010-01-04,6.431897,6.695000,81.510002,1117.699951,15.536651,38.050671,23.211447,0.423884,1.592667
2010-01-05,6.443018,6.734500,81.769997,1118.099976,15.468233,38.050671,23.218939,0.430073,1.592667
2010-01-06,6.340532,6.612500,83.180000,1135.900024,15.078297,38.050671,23.076441,0.432824,1.592667
2010-01-07,6.328812,6.500000,82.660004,1133.099976,14.727283,38.050671,22.836458,0.424342,1.592667
2010-01-08,6.370886,6.676000,82.750000,1138.199951,14.923613,38.050671,22.993948,0.425259,1.592667
...,...,...,...,...,...,...,...,...,...
2024-12-23,254.655716,225.059998,69.239998,2612.300049,195.766968,599.316772,433.583038,139.657150,430.600006
2024-12-24,257.578674,229.050003,70.099998,2620.000000,197.345184,607.209778,437.647369,140.207108,462.279999
2024-12-26,258.396667,227.050003,69.620003,2638.800049,196.875717,602.813660,436.432068,139.917130,454.130005


In [4]:
def create_windows(data, window_size=101, max_samples=3000):
    series = []
    for i in range(0, len(data)-window_size+1, 1):
        window = data.iloc[i:i+window_size].T
        series.append(window.values)
        if len(series) == max_samples:
            break
    return np.array(series)

def inject_random_nans(X, nan_ratio=0.05, seed=42):
    """
    Inserta NaNs aleatoriamente en un porcentaje de los datos en X.
    nan_ratio: proporción de valores que serán convertidos a NaN (e.g., 0.05 = 5%)
    """
    np.random.seed(seed)
    X_nans = X.copy()
    num_samples, num_series, time_steps = X_nans.shape
    total_values = num_samples * num_series * time_steps
    num_nans = int(nan_ratio * total_values)

    # Seleccionar índices planos para asignar NaNs
    flat_indices = np.random.choice(total_values, num_nans, replace=False)

    # Convertir índices planos a 3D
    sample_idx, series_idx, time_idx = np.unravel_index(flat_indices, X_nans.shape)
    X_nans[sample_idx, series_idx, time_idx] = np.nan

    return X_nans

In [5]:
# escalamiento de datos para que estén entre 0 y 1
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, index=data.index, columns=data.columns)

series = create_windows(data_scaled)

X = series[:, :, :100]
X = inject_random_nans(X)
X = np.nan_to_num(X, nan=0.0)  # reemplazar nan por 0

Y = series[:, :, 100:]

print("Datos preparados:")
print("X shape:", X.shape)
print("Y shape:", Y.shape)


Datos preparados:
X shape: (3000, 9, 100)
Y shape: (3000, 9, 1)


In [6]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Asumimos que tienes: 
# X → (3000, 9, 100), Y → (3000, 9, 1)

# Transponer para que sea (batch, time, features)
X_tf = np.transpose(X, (0, 2, 1))  # (3000, 100, 9)
Y_tf = Y.squeeze(-1)  # (3000, 9)

# Dividir en entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X_tf, Y_tf, test_size=0.2, random_state=42)


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

def transformer(inputs, head_sz=64, num_heads=3, ff_dim=128, dropout=0.1):
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_sz)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    x_ff = layers.Dense(ff_dim, activation='relu')(x)
    x_ff = layers.Dense(inputs.shape[-1])(x_ff)
    x = layers.Dropout(dropout)(x_ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + x_ff)
    return x

def build_model(input_shape, num_layers=3, num_heads=3, ff_dim=128):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    for _ in range(num_layers):
        x = transformer(x, num_heads=num_heads, ff_dim=ff_dim)

    x = layers.Lambda(lambda x: x[:, -1, :])(x)
    outputs = layers.Dense(9)(x)
    return tf.keras.Model(inputs, outputs)

In [12]:
model = build_model(input_shape=(100, 9), num_layers=4, num_heads=5)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
60/60 [==============================] - 17s 215ms/step - loss: 0.1183 - mse: 0.1183 - val_loss: 0.0367 - val_mse: 0.0367
Epoch 2/20
60/60 [==============================] - 13s 211ms/step - loss: 0.0496 - mse: 0.0496 - val_loss: 0.0138 - val_mse: 0.0138
Epoch 3/20
60/60 [==============================] - 13s 212ms/step - loss: 0.0296 - mse: 0.0296 - val_loss: 0.0068 - val_mse: 0.0068
Epoch 4/20
60/60 [==============================] - 13s 214ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.0048 - val_mse: 0.0048
Epoch 5/20
60/60 [==============================] - 13s 213ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.0045 - val_mse: 0.0045
Epoch 6/20
60/60 [==============================] - 13s 213ms/step - loss: 0.0186 - mse: 0.0186 - val_loss: 0.0038 - val_mse: 0.0038
Epoch 7/20
60/60 [==============================] - 13s 217ms/step - loss: 0.0167 - mse: 0.0167 - val_loss: 0.0039 - val_mse: 0.0039
Epoch 8/20
60/60 [==============================] - 13s 215ms/step - 

In [14]:
loss, mse = model.evaluate(X_test, Y_test, verbose=0)

print(f"MSE en test: {loss:.6f}")

MSE en test: 0.001758


In [16]:
baseline_pred = X_test[:, -1, :]
baseline_loss = tf.reduce_mean(tf.square(baseline_pred - Y_test)).numpy()

print(f"Loss predictor trivial: {baseline_loss:.6f}")
print(f"Loss transformer entrenado: {loss:.6f}")

Loss predictor trivial: 0.004415
Loss transformer entrenado: 0.001758
